In [ ]:
import pandas as pd
import datetime

In [ ]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

In [ ]:
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"]) #formato fecha
df_vp["mes"]  = df_vp["fecha"].dt.month #agregar mes
df_vp["ano"]  = df_vp["fecha"].dt.year #agregar año
df_vp["dia"]  = df_vp["fecha"].dt.day #agregar año


pd.set_option("display.max_rows", None)   # muestra todas las filas
pd.set_option("display.max_colwidth", None)  # muestra texto completo en celdas

df_perdidas = df_vp[df_vp["cantidad"] > df_vp["cantidad_existencia"]] # ventas peridas realmente
df_perdidas = df_perdidas.drop(["product_id_num","store_id","warehouse_id","cliente_id","id","cliente_id_num","warehouse_id_num","warehouse_name"],axis=1) #eliminar columnas que no sirven
df_perdidas["vp"] = df_vp["cantidad"] - df_vp["cantidad_existencia"]


# REGLAS FLTRADO DE VENTAS PERDIDAS
vp_filtradas = df_perdidas[(df_perdidas["cantidad"]>=1)  & (df_perdidas["cantidad"]<=99)].copy()
vp_filtradas.sort_values(by=["vp"], ascending=False).head(5)

vp_filtradas = vp_filtradas[~vp_filtradas["cliente_name"].str.contains("PROMOCION", case=False, na=False)].copy() #Solicitar nueva regla retirar ventas perdidas para cliente promocion esta duplicandolas

vp_def = vp_filtradas[["store_name","product_id","origen","cliente_name","vp","mes","dia","ano"]]
vp_def.head()

In [ ]:
import ast

def get_second(val):
    try:
        # convierte el string en lista real de Python
        lista = ast.literal_eval(str(val))
        if isinstance(lista, list) and len(lista) > 1:
            return lista[1]  # devuelve el segundo elemento
    except Exception:
        return None
    return None

vp_def["product_id"] = vp_def["product_id"].apply(get_second)

import re

def split_cliente(val):
    if val is None:
        return None, None
    text = str(val)
    match = re.match(r"\[(.*?)\]\s*(.*)", text)
    if match:
        return match.group(1), match.group(2)  # (ID, Nombre)
    return None, text

# Crear dos columnas nuevas
vp_def = vp_def.copy()

vp_def.loc[:, "cliente_id"], vp_def.loc[:, "cliente_name"] = zip(*vp_def["cliente_name"].apply(split_cliente))
vp_def = vp_def.drop(columns=["cliente_id"])

vp_def.head()

In [ ]:
import re

def split_producto(val):
    if val is None:
        return None, None, None
    text = str(val).strip()
    
    # buscar con regex: [producto] descripcion (codigo)
    match = re.match(r"\[(.*?)\]\s*(.*)\((.*?)\)\s*$", text)
    if match:
        producto = match.group(1).strip()
        descripcion = match.group(2).strip().rstrip(",")   # quitamos coma final si la hay
        codigo_interno = match.group(3).strip()
        return producto, descripcion, codigo_interno
    
    return None, text, None  # fallback: si no cumple el patrón

# aplicar y crear nuevas columnas
vp_def.loc[:, ["producto", "descripcion", "codigo_interno"]] = list(vp_def["product_id"].apply(split_producto))
vp_def = vp_def.drop(columns=["product_id"], errors="ignore")

vp_def


In [ ]:
# Guardar en un archivo Excel
#vp_def.to_excel("vp_filtradas.xlsx", index=False)